In [2]:
import pandas as pd

datapath ="C:/Users/garim/Documents/Capstone/Full data/Normalized/rna_norm_counts.csv"
rna = pd.read_csv(datapath)


In [4]:
tss_map = pd.read_excel("C:/Users/garim/Documents/Capstone/Full data/tss_map.xlsx",sheet_name='tss_map',header=0,converters={'tssC':str,'TissueSourceSite':str}, na_values=[],keep_default_na  = False)
sample_map = pd.read_excel("C:/Users/garim/Documents/Capstone/Full data/sample_map.xlsx",sheet_name='sample_map',header=0,converters={'sampleC':str,'sample':str}, na_values=[], keep_default_na  = False)

In [ ]:
trna = rna.transpose()
header = trna.iloc[0]
trna = trna[1:]
trna.columns = header

Let's look at a chunk of the miRNA data which is otherwise huge:

In [ ]:
trna.shape

Extracting study name to split the data by study

In [ ]:
trna['study'] = trna.index.str.split('.').str[0]

In [ ]:
trna.iloc[60:70,]

In [ ]:
trna['study'].value_counts()

In [ ]:
trna_target = trna[trna['study']=="TARGET"]
trna_target.shape

In [ ]:
trna_tcga = trna[trna['study']=="TCGA"]
trna_tcga.shape

In [ ]:
max(trna_tcga.index.str.len()) == min(trna_tcga.index.str.len())

Splitting Patient ID into components to obtain Dosease and sample information

In [ ]:
trna_tcga[['study_1C','tssC','participant','sample_vialC','portion_analyte','plate','center']] = pd.DataFrame(trna_tcga.index.str.split('.').values.tolist(), index= tmirna_tcga.index)
trna_tcga.iloc[0:5,:]

In [ ]:
trna_tcga['sampleC'] =  trna_tcga['sample_vialC'].str.slice(0,2)
trna_tcga['vial']= trna_tcga['sample_vialC'].str.slice(2,3)
trna_tcga['portion'] =  trna_tcga['portion_analyte'].str.slice(0,2)
trna_tcga['analyte']= trna_tcga['portion_analyte'].str.slice(2,3)

trna_tcga.iloc[0:5,:]

Pulling the meaning of TSS and Sample codes under 'tss' and 'sample' columns respectively

In [ ]:
trna_tcga = pd.merge(trna_tcga, tss_map, how = 'left', on = 'tssC')
trna_tcga = pd.merge(trna_tcga, sample_map, how = 'left', on = 'sampleC')
trna_tcga=trna_tcga.drop(['study_1C'], axis = 1)

In [ ]:
rna_count_summary = trna_tcga.groupby(['TissueSourceSite','sample','portion','sampleC'])['index'].count()
rna_count_summary.to_csv("C:/Users/garim/Documents/Capstone/Full data/Normalized/rna_count_summary.csv")

Exporting the modified TCGA miRNA dataframe to TmiRNA_TCGA csv file

In [ ]:
trna_tcga.to_csv("C:/Users/garim/Documents/GitHub/Precision-Medicine-for-Cancer/Full data/TRNA_TCGA.csv")